In [1]:
import numpy as np
import matplotlib.pylab as plt
import pickle

from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

本章的主题是卷积神经网络（Convolutional Neural Network，CNN）。

CNN被用于图像识别、语音识别等各种场合，在图像识别的比赛中，基于深度学习的方法几乎都以CNN为基础。

# 总结

本章所学的内容

- CNN在此前的全连接层的网络中新增了卷积层和池化层。
- 使用im2col函数可以简单、高效地实现卷积层和池化层。
- 通过CNN的可视化，可知随着层次变深，提取的信息愈加高级。
- LeNet和AlexNet是CNN的代表性网络。
- 在深度学习的发展中，大数据和GPU做出了很大的贡献。


# 整体结构

CNN和之前介绍的神经网络一样，可以像乐高积木一样通过组装层来构建。不过，CNN中新出现了卷积层（Convolution层）和池化层（Pooling层）。

<img src="img/7_1.png" alt="Drawing" style="width: 500px;"/>

- CNN 的层的连接顺序是“Convolution - ReLU -（Pooling）”（Pooling层有时会被省略）。

- 这可以理解为之前的“Affine - ReLU”连接被替换成了“Convolution - ReLU -（Pooling）”连接。

- 还需要注意的是，在图7-2的CNN中，靠近输出的层中使用了之前的“Affine - ReLU”组合。

- 此外，最后的输出层中使用了之前的“Affine - Softmax”组合。

这些都是一般的CNN中比较常见的结构。


# 卷积层

**为什么要卷积层？**

之前介绍的全连接的神经网络中使用了全连接层（Affine层）。在全连接层中，相邻层的神经元全部连接在一起。

但是，全连接层存在什么问题呢？那就是数据的形状被“忽视”了。

比如，输入数据是图像时，图像通常是高、长、通道方向上的3维形状。但是，向全连接层输入时，需要将3维数据拉平为1维数据。

前面提到的使用了MNIST数据集的例子中，输入图像就是1通道、高28像素、长28像素的（1, 28, 28）形状，但却被排成1列，以784个数据的形式输入到最开始的Affine层。

图像是3维形状，这个形状中应该含有重要的空间信息。

3维形状中可能隐藏有值得提取的本质模式。但是，因为全连接层会忽视形状，将全部的输入数据作为相同的神经元（同一维度的神经元）处理，所以无法利用与形状相关的信息。

**而卷积层可以保持形状不变。当输入数据是图像时，卷积层会以3维数据的形式接收输入数据，并同样以3维数据的形式输出至下一层。因此，在CNN中，可以（有可能）正确理解图像等具有形状的数据。**

## 卷积运算

卷积层进行的处理就是卷积运算。卷积运算相当于图像处理中的“滤波器运算”。

<img src="img/7_3.png" alt="Drawing" style="width: 500px;"/>

如图7-3所示，卷积运算对输入数据应用滤波器。在这个例子中，输入数据是有高长方向的形状的数据，滤波器也一样，有高长方向上的维度。假设用（height, width）表示数据和滤波器的形状，则在本例中，输入大小是(4, 4)，滤波器大小是(3, 3)，输出大小是(2, 2)。另外，有的文献中也会用“核”这个词来表示这里所说的“滤波器”。

对于输入数据，卷积运算以一定间隔滑动滤波器的窗口并应用。
将各个位置上滤波器的元素和输入的对应元素相乘，然后再求和（有时将这个计算称为乘积累加运算）。然后，将这个结果保存到输出的对应位置。将这个过程在所有位置都进行一遍，就可以得到卷积运算的输出。

<img src="img/7_4.png" alt="Drawing" style="width: 500px;"/>


CNN中也存在偏置。
如图7-5所示，向应用了滤波器的数据加上了偏置。偏置通常只有1个 （1 × 1）（本例中，相对于应用了滤波器的4个数据，偏置只有1个），这个值会被加到应用了滤波器的所有元素上。
<img src="img/7_5.png" alt="Drawing" style="width: 500px;"/>



## 填充

- 在进行卷积层的处理之前，有时要向输入数据的周围填入固定的数据（比如0等），这称为填充（padding），是卷积运算中经常会用到的处理。

- 使用填充主要是为了调整输出的大小。比如图7-5，对大小为(4, 4)的输入数据应用(3, 3)的滤波器时，输出大小变为(2, 2)，相当于输出大小比输入大小缩小了 2个元素。这在反复进行多次卷积运算的深度网络中会成为问题。为什么呢？因为如果每次进行卷积运算都会缩小空间，那么在某个时刻输出大小就有可能变为 1，导致无法再应用卷积运算。为了避免出现这样的情况，就要使用填充。

<img src="img/7_6.png" alt="Drawing" style="width: 500px;"/>
在图7-6的例子中，对大小为(4, 4)的输入数据应用了幅度为1的填充。“幅度为1的填充”是指用幅度为1像素的0填充周围。

通过填充，大小为(4, 4)的输入数据变成了(6, 6)的形状。然后，应用大小为(3, 3)的滤波器，生成了大小为(4, 4)的输出数据。这个例子中将填充设成了1，不过填充的值也可以设置成2、3等任意的整数。在图7-5的例子中，如果将填充设为2，则输入数据的大小变为(8, 8)；如果将填充设为3，则大小变为(10, 10)。

## 步幅

应用滤波器的位置间隔称为步幅（stride）。之前的例子中步幅都是1，如果将步幅设为2，则如图7-7所示，应用滤波器的窗口的间隔变为2个元素。
<img src="img/7_7.png" alt="Drawing" style="width: 500px;"/>




**综上，**
- 增大步幅后，输出大小会变小。
- 而增大填充后，输出大小会变大。

## 三维数据的卷积运算

对于3维数据，除了高、长方向之外，还需要处理通道方向。以下为具体的卷积运算的例子。

<img src="img/7_8.png" alt="Drawing" style="width: 500px;"/>

和2维数据时（图7-3的例子）相比，可以发现纵深方向（通道方向）上特征图增加了。通道方向上有多个特征图时，会按通道进行输入数据和滤波器的卷积运算，并将结果相加，从而得到输出。

需要注意的是，在3维数据的卷积运算中，输入数据和滤波器的通道数要设为相同的值。

在这个例子中，输入数据和滤波器的通道数一致，均为3。
滤波器大小可以设定为任意值（不过，每个通道的滤波器大小要全部相同）。这个例子中滤波器大小为(3, 3)，但也可以设定为(2, 2)、(1, 1)、(5, 5)等任意值。

<img src="img/7_9.png" alt="Drawing" style="width: 500px;"/>




# 池化层

**为什么要有池化层？**

- 这一层有时候确实会被省略。

**先来看看实现**，为什么存在现在我还回答不来~

- 池化是缩小高、长方向上的空间的运算。比如，如图7-14所示，进行将2 × 2的区域集约成1个元素的处理，缩小空间大小。
- 下图的运算展示的是 “Max池化”，如图所示，从2 × 2的区域中取出最大的元素。

<img src="img/7_14.png" alt="Drawing" style="width: 500px;"/>

- 此外，这个例子中将步幅设为了2，所以2 × 2的窗口的移动间隔为2个元素。
- 另外，一般来说，池化的窗口大小会和步幅设定成相同的值。比如，3 × 3的窗口的步幅会设为3，4 × 4的窗口的步幅会设为4等。
- 除了Max池化之外，还有Average池化等。相对于Max池化是从目标区域中取出最大值，Average池化则是计算目标区域的平均值。在图像识别领域，主要使用Max池化。后文说的“池化层”，均指的是Max池化。


**池化层的特征**

1. 没有要学习的参数

池化层和卷积层不同，没有要学习的参数。池化只是从目标区域中取最大值（或者平均值），所以不存在要学习的参数。

2. 通道数不发生变化

经过池化运算，输入数据和输出数据的通道数不会发生变化。如图7-15所示，计算是按通道独立进行的。
<img src="img/7_15.png" alt="Drawing" style="width: 500px;"/>


3. 对微小的位置变化具有鲁棒性（健壮）

输入数据发生微小偏差时，池化仍会返回相同的结果。因此，池化对输入数据的微小偏差具有鲁棒性。比如，3 × 3的池化的情况下，如图7-16所示，池化会吸收输入数据的偏差（根据数据的不同，结果有可能不一致）。

<img src="img/7_16.png" alt="Drawing" style="width: 500px;"/>


# 卷积层和池化层的实现

实际上，通过使用 im2col技巧，就可以很轻松地实现。

本节将介绍这种技巧，将问题简化，然后再进行卷积层的实现。

## 基于im2col展开

im2col这个名称是“image to column”的缩写，翻译过来就是“从图像到矩阵”的意思。Caffe、Chainer 等深度学习框架中有名为im2col的函数，并且在卷积层的实现中，都使用了im2col。

im2col是一个函数，将输入数据展开以适合滤波器（权重）。

如图7-17所示，对3维的输入数据应用im2col后，数据转换为2维矩阵（正确地讲，是把包含批数量的4维数据转换成了2维数据）。

<img src="img/7_17.png" alt="Drawing" style="width: 500px;"/>


使用im2col展开输入数据后，之后就只需将卷积层的滤波器（权重）纵向展开为1列，并计算2个矩阵的乘积即可（参照图7-19）。这和全连接层的Affine层进行的处理基本相同。

<img src="img/7_19.png" alt="Drawing" style="width: 500px;"/>

基于im2col方式的输出结果是2维矩阵。因为CNN中数据会保存为4维数组，所以要将2维输出数据转换为合适的形状。以上就是卷积层的实现流程。

小结：理解下来，就是说即使im2col还是将多维数据处理成了二维，但是在这个二维数据中是包含了多维的信息的。

In [2]:
# 来看一下 im2col函数的具体实现，来自：common/util.py中
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    """
    Parameters
    ----------
    input_data : 由(数据量, 通道, 高, 长)的4维数组构成的输入数据
    filter_h : 滤波器的高
    filter_w : 滤波器的长
    stride : 步幅
    pad : 填充
    Returns
    -------
    col : 2维数组
    """
    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1

    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col


In [3]:
# 关于4维数据
# CNN中各层间传递的数据是4维数据。所谓4维数据，比如
# 数据的形状是(10, 1, 28, 28)，则它对应10个高为28、长为28、通道为1的数据。用Python来实现的话，如下所示。

import numpy as np
x = np.random.rand(10, 1, 28, 28)
x.shape

# 如果要访问第1个数据，只要写x[0]就可以了（注意Python的索
# 引是从0开始的）。同样地，用x[1]可以访问第2个数据。
x[0].shape

(10, 1, 28, 28)

(1, 28, 28)

In [4]:
x[0]

array([[[6.32643537e-01, 6.42474801e-01, 3.21670576e-01, 6.96281877e-01,
         8.59470655e-01, 2.63835100e-01, 1.50653785e-01, 3.19361312e-01,
         7.18357960e-01, 7.15669373e-01, 6.59655227e-01, 9.71216687e-01,
         2.43340125e-01, 8.80816032e-01, 6.12272002e-01, 8.65376419e-01,
         5.71428642e-01, 6.02433407e-01, 8.17676525e-01, 1.50693025e-02,
         1.62669231e-01, 9.88943746e-01, 7.80095987e-01, 7.05518784e-01,
         7.39321824e-01, 5.25150234e-01, 5.30621536e-01, 5.38705332e-01],
        [1.68222131e-01, 4.23565603e-01, 8.04093536e-01, 3.87374704e-01,
         4.85811210e-01, 2.68482560e-01, 7.19132763e-01, 8.98097098e-01,
         7.21062730e-01, 4.98239965e-01, 3.14800272e-01, 6.41085829e-01,
         6.59243791e-01, 1.75235297e-01, 5.35979080e-01, 9.17585889e-01,
         5.59160586e-01, 6.70996648e-01, 4.42676129e-02, 8.08451193e-02,
         9.59158395e-01, 9.07384048e-01, 7.59715004e-01, 7.50327521e-01,
         3.31907910e-01, 6.16314120e-02, 8.0047777

In [5]:
# 如果要访问第1个数据的第1个通道的空间数据，可以写成下面这样。
x[0, 0] # 或者x[0][0]

array([[6.32643537e-01, 6.42474801e-01, 3.21670576e-01, 6.96281877e-01,
        8.59470655e-01, 2.63835100e-01, 1.50653785e-01, 3.19361312e-01,
        7.18357960e-01, 7.15669373e-01, 6.59655227e-01, 9.71216687e-01,
        2.43340125e-01, 8.80816032e-01, 6.12272002e-01, 8.65376419e-01,
        5.71428642e-01, 6.02433407e-01, 8.17676525e-01, 1.50693025e-02,
        1.62669231e-01, 9.88943746e-01, 7.80095987e-01, 7.05518784e-01,
        7.39321824e-01, 5.25150234e-01, 5.30621536e-01, 5.38705332e-01],
       [1.68222131e-01, 4.23565603e-01, 8.04093536e-01, 3.87374704e-01,
        4.85811210e-01, 2.68482560e-01, 7.19132763e-01, 8.98097098e-01,
        7.21062730e-01, 4.98239965e-01, 3.14800272e-01, 6.41085829e-01,
        6.59243791e-01, 1.75235297e-01, 5.35979080e-01, 9.17585889e-01,
        5.59160586e-01, 6.70996648e-01, 4.42676129e-02, 8.08451193e-02,
        9.59158395e-01, 9.07384048e-01, 7.59715004e-01, 7.50327521e-01,
        3.31907910e-01, 6.16314120e-02, 8.00477770e-01, 8.31144

In [6]:
# 实际使用一下im2col函数

import sys, os
sys.path.append(os.pardir)
from common.util import im2col

x1 = np.random.rand(1, 3, 7, 7)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape) # (9, 75)

(9, 75)


In [7]:
x1[0, 0]

array([[0.04183425, 0.67447583, 0.67639601, 0.12734469, 0.82339326,
        0.67134099, 0.16753084],
       [0.57360255, 0.45914323, 0.62161179, 0.40413642, 0.89976966,
        0.54698421, 0.86986707],
       [0.95544543, 0.83314696, 0.46817484, 0.52143367, 0.69215629,
        0.29936641, 0.92434576],
       [0.91232351, 0.53149153, 0.24349561, 0.80911406, 0.3681731 ,
        0.01492813, 0.44656437],
       [0.55184151, 0.63167868, 0.88066291, 0.00553271, 0.5119636 ,
        0.44897562, 0.81828986],
       [0.42698635, 0.60062349, 0.08390255, 0.66245316, 0.09183378,
        0.70304945, 0.42403892],
       [0.92141501, 0.0749329 , 0.90604422, 0.92939369, 0.16265076,
        0.3668068 , 0.72529786]])

In [8]:
col1[0]

array([0.04183425, 0.67447583, 0.67639601, 0.12734469, 0.82339326,
       0.57360255, 0.45914323, 0.62161179, 0.40413642, 0.89976966,
       0.95544543, 0.83314696, 0.46817484, 0.52143367, 0.69215629,
       0.91232351, 0.53149153, 0.24349561, 0.80911406, 0.3681731 ,
       0.55184151, 0.63167868, 0.88066291, 0.00553271, 0.5119636 ,
       0.02593091, 0.87926522, 0.41884732, 0.48512099, 0.04020266,
       0.83890173, 0.90519112, 0.16904747, 0.86905581, 0.44468782,
       0.87443602, 0.93347631, 0.23869905, 0.52063498, 0.38291509,
       0.59238708, 0.58267194, 0.03098662, 0.52058164, 0.71361133,
       0.10001258, 0.41038667, 0.31979945, 0.43327257, 0.66582183,
       0.52405106, 0.83719731, 0.17681769, 0.35395387, 0.79579801,
       0.9034359 , 0.35926726, 0.46337699, 0.02706691, 0.34739165,
       0.01710236, 0.85482147, 0.95381532, 0.49977507, 0.56070597,
       0.79138351, 0.84498847, 0.82641562, 0.29753697, 0.03824147,
       0.78888154, 0.29610244, 0.45192329, 0.44810593, 0.98874

In [9]:
x2 = np.random.rand(10, 3, 7, 7) # 10个数据
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape) # (90, 75)

(90, 75)


## 使用im2col来实现卷积层

In [10]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        # FN、C、FH、FW分别是 Filter Number（滤波器数量）、Channel、Filter Height、Filter Width的缩写。
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2*self.pad - FW) / self.stride)
        
        # 用im2col展开输入数据，并用reshape将滤波器展开为2维数组。然后，计算展开后的矩阵的乘积。
        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T # 滤波器的展开，这里通过reshape(FN,-1)将参数指定为-1，会自动计算-1维度上的元素个数，以使多维数组的元素个数前后一致。
        out = np.dot(col, col_W) + self.b
        
        # transpose会更改多维数组的轴的顺序。如图7-20所示，通过指定从0开始的索引（编号）序列，就可以更改轴的顺序。
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)
        return out
    
    def backward(self, dout):
        FN, C, FH, FW = self.W.shape
        dout = dout.transpose(0,2,3,1).reshape(-1, FN)

        self.db = np.sum(dout, axis=0)
        self.dW = np.dot(self.col.T, dout)
        self.dW = self.dW.transpose(1, 0).reshape(FN, C, FH, FW)

        dcol = np.dot(dout, self.col_W.T)
        dx = col2im(dcol, self.x.shape, FH, FW, self.stride, self.pad)

        return dx

<img src="img/7_20.png" alt="Drawing" style="width: 500px;"/>


以上代码就是卷积层的forward处理的实现。

通过使用im2col进行展开，基本上可以像实现全连接层的Affine层一样来实现（5.6节）。

接下来是卷积层的反向传播的实现，因为和Affine层的实现有很多共通的地方，所以就不再介绍了。

但有一点需要注意，在进行卷积层的反向传播时，必须进行im2col的逆处理。这可以使用本书提供的col2im函数

## 池化层的实现

池化层的实现和卷积层相同，都使用im2col展开输入数据。不过，池化的情况下，在通道方向上是独立的，这一点和卷积层不同。

池化的应用区域按通道单独展开，顺序拼接在一起。

<img src="img/7_21.png" alt="Drawing" style="width: 500px;"/>

这样展开之后，只需对展开的矩阵求各行的最大值，并转换为合适的形状即可（图7-22）。

<img src="img/7_22.png" alt="Drawing" style="width: 500px;"/>


In [11]:
# 池化层的实现代码
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)
        
        #(1)展开：展开输入数据
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)
        
        # (2)最大值：求各行的最大值
        out = np.max(col, axis=1)
        
        # (3)转换：转换为合适的输出大小
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)
        return out
    
    def backward(self, dout):
        dout = dout.transpose(0, 2, 3, 1)
        
        pool_size = self.pool_h * self.pool_w
        dmax = np.zeros((dout.size, pool_size))
        dmax[np.arange(self.arg_max.size), self.arg_max.flatten()] = dout.flatten()
        dmax = dmax.reshape(dout.shape + (pool_size,)) 
        
        dcol = dmax.reshape(dmax.shape[0] * dmax.shape[1] * dmax.shape[2], -1)
        dx = col2im(dcol, self.x.shape, self.pool_h, self.pool_w, self.stride, self.pad)
        
        return dx


# CNN的实现

以手写数字识别的CNN为例：网络的构成是“Convolution - ReLU - Pooling -Affine - ReLU - Affine - Softmax”，我们将它实现为名为SimpleConvNet的类。

<img src="img/7_23.png" alt="Drawing" style="width: 500px;"/>




In [12]:
# CNN的代码实现

In [16]:
class SimpleConvNet:
    def __init__(self, 
                 input_dim=(1, 28, 28), # 输入数据的维度：（通道，高，长）
                 conv_param={'filter_num':30, 'filter_size':5,'pad':0, 'stride':1}, # 卷积层的超参数（字典）
                 hidden_size=100, # 隐藏层（全连接）的神经元数量
                 output_size=10, # 输出层（全连接）的神经元数量
                 weight_init_std=0.01 # 初始化时权重的标准差
                ): 
        
        # 第一部分：初始化最开始。计算卷积层和池化层的输出大小
        filter_num = conv_param['filter_num'] # 滤波器的数量
        filter_size = conv_param['filter_size'] # 滤波器的大小
        filter_pad = conv_param['pad'] # 填充
        filter_stride = conv_param['stride'] # 步幅
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad) / \
        filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2) *(conv_output_size/2))
        
        # 第二部分：权重参数的初始化。
        # 学习所需的参数是第1层的卷积层和剩余两个全连接层的权重和偏置，将这些参数保存在实例变量的params字典中。
        self.params['W1'] = weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)
        
        # 第三部分：生成必要的层。
        # 从最前面开始按顺序向有序字典（OrderedDict）的layers中添加层。只有最后的SoftmaxWithLoss层被添加到别的变量lastLayer中。
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'],self.params['b1'],conv_param['stride'],conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'],self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'],self.params['b3'])
        self.last_layer = softmaxwithloss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x

    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    # 基于误差反向传播法求梯度的代码实现
    def gradient(self, x, t):
        # forward
        self.loss(x, t)
        
        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
        
        # 设定
        grads = {}
        grads['W1'] = self.layers['Conv1'].dW
        grads['b1'] = self.layers['Conv1'].db
        grads['W2'] = self.layers['Affine1'].dW
        grads['b2'] = self.layers['Affine1'].db
        grads['W3'] = self.layers['Affine2'].dW
        grads['b3'] = self.layers['Affine2'].db
        return grads
  

**总结：**

卷积层和池化层是图像识别中必备的模块。CNN可以有效读取图像中的某种特性，在手写数字识别中，还可以实现高精度的识别。

# CNN中到底进行了什么处理？

本节将通过卷积层的可视化，探索CNN中到底进行了什么处理。

图7-24中，学习前的滤波器是随机进行初始化的，所以在黑白的浓淡上没有规律可循，但学习后的滤波器变成了有规律的图像。

<img src="img/7_24.png" alt="Drawing" style="width: 500px;"/>

如果要问图7-24中右边的有规律的滤波器在“观察”什么，答案就是它在观察边缘（颜色变化的分界线）和斑块（局部的块状区域）等。比如，左半部分为白色、右半部分为黑色的滤波器的情况下，如图7-25所示，会对垂直方向上的边缘有响应。

<img src="img/7_25.png" alt="Drawing" style="width: 500px;"/>

图7-25中显示了选择两个学习完的滤波器对输入图像进行卷积处理时的结果。我们发现“滤波器1”对垂直方向上的边缘有响应，“滤波器2”对水平方向上的边缘有响应。由此可知，卷积层的滤波器会提取边缘或斑块等原始信息。而刚才实现的CNN会将这些原始信息传递给后面的层。

上面的结果是针对第1层的卷积层得出的。第1层的卷积层中提取了边缘或斑块等“低级”信息，那么在堆叠了多层的CNN中，各层中又会提取什么样的信息呢？根据深度学习的可视化相关的研究[17][18]，随着层次加深，提取的信息（正确地讲，是反映强烈的神经元）也越来越抽象。

图7-26中展示了进行一般物体识别（车或狗等）的8层CNN。这个网络结构的名称是下一节要介绍的AlexNet。AlexNet网络结构堆叠了多层卷积层和池化层，最后经过全连接层输出结果。图7-26的方块表示的是中间数据，对于这些中间数据，会连续应用卷积运算。

<img src="img/7_26.png" alt="Drawing" style="width: 500px;"/>


如图7-26所示，如果堆叠了多层卷积层，则随着层次加深，提取的信息也愈加复杂、抽象，这是深度学习中很有意思的一个地方。最开始的层对简单的边缘有响应，接下来的层对纹理有响应，再后面的层对更加复杂的物体部件有响应。也就是说，随着层次加深，神经元从简单的形状向“高级”信息变化。换句话说，就像我们理解东西的“含义”一样，响应的对象在逐渐变化。

# 具有代表性的 CNN

## LeNet

在1998年首次被提出的CNN元祖LeNet

LeNet在1998年被提出，是进行手写数字识别的网络。如图7-27所示，它有连续的卷积层和池化层（正确地讲，是只“抽选元素”的子采样层），最后经全连接层输出结果。

<img src="img/7_27.png" alt="Drawing" style="width: 500px;"/>

和“现在的CNN”相比，LeNet有几个不同点。
- 第一个不同点在于激活函数。LeNet中使用sigmoid函数，而现在的CNN中主要使用ReLU函数。
- 此外，原始的LeNet中使用子采样（subsampling）缩小中间数据的大小，而现在的CNN中Max池化是主流。

综上，LeNet与现在的CNN虽然有些许不同，但差别并不是那么大。想到LeNet是20多年前提出的最早的CNN，还是很令人称奇的。

## AlexNet

是在深度学习受到关注的2012年被提出的AlexNet

在LeNet问世20多年后，AlexNet被发布出来。AlexNet是引发深度学习热潮的导火线，不过它的网络结构和LeNet基本上没有什么不同，如图7-28所示。

<img src="img/7_28.png" alt="Drawing" style="width: 500px;"/>

AlexNet叠有多个卷积层和池化层，最后经由全连接层输出结果。虽然结构上AlexNet和LeNet没有大的不同，但有以下几点差异。
- 激活函数使用ReLU。 
- 使用进行局部正规化的LRN（Local Response Normalization）层。
- 使用Dropout（6.4.3节）。


大多数情况下，深度学习（加深了层次的网络）存在大量的参数。因此，学习需要大量的计算，并且需要使那些参数“满意”的大量数据。

可以说是 GPU和大数据给这些课题带来了希望。